In [15]:
from astropy.io import fits
from astropy.table import Table
import random
import numpy as np

In [24]:
# with fits.open('gll_psc_v16.fit') as hdu:
#     FGL3 = Table(hdu[1].data)
#     headers = hdu[0].header
#     cols = hdu[1].columns
#     data_3FGL = hdu[1].data
 
# cols.info()
# print(FGL3)
# # print(data_3FGL[0][1])
# # print(data_3FGL['Source_Name'])
# # print(data_3FGL[0])
# #print(data_3FGL[2]['CLASS1'])
# pulsars = []
# agns = []
# others = []
# unclass = []
# for i in range(len(data_3FGL)): #need to find out which classes count for AGNs and pulsars
#     #pulsars (classification labels in 3FGL: PSR, psr)
#     if data_3FGL[i]['CLASS1'] == 'PSR' or data_3FGL[i]['CLASS1'] == 'psr':
#         pulsars.append(data_3FGL[i])
#     #AGNs (classification labels in the 3FGL catalog: agn, FSRQ, fsrq, BLL, bll, BCU, bcu, RDG, rdg, NLSY1, nlsy1, ssrq, and sey)
#     elif data_3FGL[i]['CLASS1'] == 'agn' or data_3FGL[i]['CLASS1'] == 'FSRQ' or data_3FGL[i]['CLASS1'] == 'fsrq' or data_3FGL[i]['CLASS1'] == 'BLL' or data_3FGL[i]['CLASS1'] == 'bll' or data_3FGL[i]['CLASS1'] == 'BCU' or data_3FGL[i]['CLASS1'] == 'bcu' or data_3FGL[i]['CLASS1'] == 'RDG' or data_3FGL[i]['CLASS1'] == 'rdg' or data_3FGL[i]['CLASS1'] == 'NLSY1' or data_3FGL[i]['CLASS1'] == 'nlsy1' or data_3FGL[i]['CLASS1'] == 'ssrq' or data_3FGL[i]['CLASS1'] == 'sey':
#         agns.append(data_3FGL[i])
#     elif data_3FGL[i]['CLASS1'] == '':
#         others.append(data_3FGL[i])
#     else:
#         unclass.append(data_3FGL[i])


In [27]:
hdul_3fgl = fits.open('gll_psc_v16.fit') #add file directory data/

classes = hdul_3fgl[1].data['CLASS1']
agn_classes = ['psr  ', 'agn  ', 'FSRQ ', 'fsrq ', 'BLL  ', 'bll  ', 'BCU  ', 'bcu  ', 'RDG  ', 'rdg  ', 'NLSY1', 'nlsy1', 'ssrq ', 'sey  ']
pulsar_classes = ['PSR  ', 'psr  ']
no_class = '     '
agn_mask = np.isin(classes, agn_classes)
pulsar_mask = np.isin(classes, pulsar_classes)
noclass_mask = classes == no_class

[False False False ...  True  True  True]


In [46]:
oversample = sum(agn_mask) - sum(pulsar_mask)

duplicated = []
for i in range(classes.size):
    duplicated.append(hdul_3fgl[1].data[i])
    
pulsars = []
for i in range(classes.size):
    #pulsars (classification labels in 3FGL: PSR, psr)
    if hdul_3fgl[1].data[i]['CLASS1'] == 'PSR' or data_3FGL[i]['CLASS1'] == 'psr':
        pulsars.append(data_3FGL[i])
    
for i in range(oversample):
    duplicated.append(random.choice(pulsars))
    
duplicated = np.array(duplicated)
    
# print(overfit)
# print(len(data_3FGL))
# print(len(pulsars))
# print(len(agns))
# print(len(over_3FGL))
print(classes.size)
print(oversample)
print(len(duplicated))

3034
1601
4635
